In [4]:
import numpy as np
import pandas as pd

from itertools import combinations_with_replacement

from sklearn.metrics import r2_score
from sklearn.datasets import make_regression

# Common Regression class

In [5]:
class Regression:
    def __init__(self, learning_rate, iteration, regularization):
        """
        :param learning_rate: A samll value needed for gradient decent, default value id 0.1.
        :param iteration: Number of training iteration, default value is 10,000.
        """
        self.m = None
        self.n = None
        self.w = None
        self.b = None
        self.regularization = regularization # will be the l1/l2 regularization class according to the regression model.
        self.lr = learning_rate
        self.it = iteration

    def cost_function(self, y, y_pred):
        """
        :param y: Original target value.
        :param y_pred: predicted target value.
        """
        return (1 / (2*self.m)) * np.sum(np.square(y_pred - y)) + self.regularization(self.w)
    
    def hypothesis(self, weights, bias, X):
        """
        :param weights: parameter value weight.
        :param X: Training samples.
        """
        return np.dot(X, weights) #+ bias

    def train(self, X, y):
        """
        :param X: training data feature values ---> N Dimentional vector.
        :param y: training data target value -----> 1 Dimentional array.
        """
        # Insert constant ones for bias weights.
        X = np.insert(X, 0, 1, axis=1)

        # Target value should be in the shape of (n, 1) not (n, ).
        # So, this will check that and change the shape to (n, 1), if not.
        try:
            y.shape[1]
        except IndexError as e:
            # we need to change it to the 1 D array, not a list.
            print("ERROR: Target array should be a one dimentional array not a list"
                  "----> here the target value not in the shape of (n,1). \nShape ({shape_y_0},1) and {shape_y} not match"
                  .format(shape_y_0 = y.shape[0] , shape_y = y.shape))
            return 
        
        # m is the number of training samples.
        self.m = X.shape[0]
        # n is the number of features.
        self.n = X.shape[1]

        # Set the initial weight.
        self.w = np.zeros((self.n , 1))

        # bias.
        self.b = 0

        for it in range(1, self.it+1):
            # 1. Find the predicted value through the hypothesis.
            # 2. Find the Cost function value.
            # 3. Find the derivation of weights.
            # 4. Apply Gradient Decent.
            y_pred = self.hypothesis(self.w, self.b, X)
            #print("iteration",it)
            #print("y predict value",y_pred)
            cost = self.cost_function(y, y_pred)
            #print("Cost function",cost)
            # fin the derivative.
            dw = (1/self.m) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.w)
            #print("weights derivation",dw)
            #db = -(2 / self.m) * np.sum((y_pred - y))

            # change the weight parameter.
            self.w = self.w - self.lr * dw
            #print("updated weights",self.w)
            #self.b = self.b - self.lr * db


            if it % 10 == 0:
                print("The Cost function for the iteration {}----->{} :)".format(it, cost))
    def predict(self, test_X):
        """
        :param test_X: feature values to predict.
        """
        # Insert constant ones for bias weights.
        test_X = np.insert(test_X, 0, 1, axis=1)

        y_pred = self.hypothesis(self.w, self.b, test_X)
        return y_pred

# Data Creation

In [6]:
# Define the traning data.
X, y = make_regression(n_samples=50000, n_features=8)

# Chnage the shape of the target to 1 dimentional array.
y = y[:, np.newaxis]

print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 50000
Number of training features --------> 8
Shape of the target value ----------> (50000, 1)


In [7]:
# display the data.
data = pd.DataFrame(X)
data.head()


,0,1,2,3,4,5,6,7
0,-0.857270,-1.073422,-0.085756,-1.114568,-0.283575,0.139347,-0.141859,-0.123514
1,0.910372,-0.958207,-0.518141,-0.730059,0.601583,-1.012509,-0.722968,1.511905
2,-1.075336,0.878671,1.151659,0.227459,-1.815386,0.067162,0.383429,-1.900678
3,-0.159892,-2.346193,0.028133,-1.292739,-0.047946,-0.565455,0.291282,-0.632634
4,-1.502441,-0.293354,1.033080,-0.478827,-0.525721,-0.647145,0.714062,-1.624335


In [8]:
# display the data.
data_y = pd.DataFrame(y)
data_y.head()

,0
0,-143.605801
1,-237.409740
2,112.838794
3,-266.666499
4,-10.782274


# Polynomial Regression from Scratch

In [9]:
def PolynomialFeature(X, degree):
    """
    It is type of feature engineering ---> adding some more features based on the exisiting features 
    by squaring or cubing.
    :param X: data need to be converted.
    :param degree: int- The degree of the polynomial that the features X will be transformed to.
    """
    n_samples, n_features = X.shape

    # get the index combinations.
    combination = [combinations_with_replacement(range(n_features), i) for i in range(0, degree + 1)]
    combination_index = [index for obj in combination for index in obj]

    # generate a empty array with new shape.
    new_n_features = len(combination_index)
    X_new = np.empty((n_samples, new_n_features))

    for i, com_index  in enumerate(combination_index):
        X_new[:, i] = np.prod(X[:, com_index], axis=1)
    
    return X_new

# Used for Polynomial Ridge regression.
class l2_regularization:
    """Regularization used for Ridge Regression"""
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        "This will be retuned when we call this class."
        return self.lamda * np.sum(np.square(weights))
    
    def derivation(self, weights):
        "Derivation of the regulariozation function."
        return self.lamda * 2 * (weights)


In [10]:
class PolynamialRegression(Regression):
    """
    Polynomail Regression is also a type of non-linear regression with no regularization. 
    Before fitting the linear regression, the dependant variable is tranformed to some polynomail degree.
    This is basincally transforming linear data to have some nonliniarity.
    """
    def __init__(self, learning_rate, iteration, degree):
        """
        :param learning_rate: [range from 0 to infinity] the stpe distance used while doing gradiant decent.
        :param iteration: int - Number of iteration to do.
        :param degree: int - The degree of the polynomial that the feature transformed to.
        """
        self.degree = degree
        # No regularization here. So, making the regularization methods to return 0.
        self.regularization = lambda x: 0
        self.regularization.derivation = lambda x: 0
        super().__init__(learning_rate, iteration, self.regularization)
    
    def train(self, X, y):
        """
        :param X: training data feature values ---> N Dimentional vector.
        :param y: training data target value -----> 1 Dimentional array.
        """
        # change the data to 
        X_poly = PolynomialFeature(X, degree=self.degree)
        return super().train(X_poly, y)
    
    def predict(self, test_X):
        """
        :param test_X: feature values to predict.
        """
        test_X_poly = PolynomialFeature(test_X, degree=self.degree)
        return super().predict(test_X_poly)

In [11]:
#define the parameters
param = {
    "degree" : 2,
    "learning_rate" : 0.1,
    "iteration" : 100,
}
print("="*100)
polynomial_reg = PolynamialRegression(**param)

# Train the model.
polynomial_reg.train(X, y) 

# Predict the values.
y_pred = polynomial_reg.predict(X)

#Root mean square error.
score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

The Cost function for the iteration 10----->2524.546198902789 :)
The Cost function for the iteration 20----->313.8199639696676 :)
The Cost function for the iteration 30----->39.17839267886082 :)
The Cost function for the iteration 40----->4.916567388701627 :)
The Cost function for the iteration 50----->0.6225340983364702 :)
The Cost function for the iteration 60----->0.08070495018731812 :)
The Cost function for the iteration 70----->0.011282742313695108 :)
The Cost function for the iteration 80----->0.0019608909310563647 :)
The Cost function for the iteration 90----->0.0005118599780978334 :)
The Cost function for the iteration 100----->0.00019559828225020284 :)
The r2_score of the trained model 0.9999999891242503


# Polynomial Regression using scikit-learn for comparision

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# data is already defined, going to use the same data for comparision.
print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))


Number of training data samples-----> 50000
Number of training features --------> 8


In [15]:
linear_reg_sklearn = LinearRegression()

poly = PolynomialFeatures(degree = 2)
X_new = poly.fit_transform(X)
linear_reg_sklearn.fit(X, y)

# predict the value
y_pred_sklearn = linear_reg_sklearn.predict(X)
score = r2_score(y, y_pred_sklearn)
print("="*100)
print("R2 score of the model is {}".format(score))

R2 score of the model is 1.0


# Polynomial Ridge Regression from scratch

In [16]:
class PolynamialRidgeRegression(Regression):
    """
    Polynomail Ridge Regression is basically polynomial regression with l2 regularization.
    """
    def __init__(self, learning_rate, iteration, degree, lamda):
        """
        :param learning_rate: [range from 0 to infinity] the stpe distance used while doing gradiant decent.
        :param iteration: int - Number of iteration to do.
        :param degree: int - The degree of the polynomial that the feature transformed to.
        """
        self.degree = degree
        # No regularization here. So, making the regularization methods to return 0.
        self.regularization = l2_regularization(lamda)
        super().__init__(learning_rate, iteration, self.regularization)
    
    def train(self, X, y):
        """
        :param X: training data feature values ---> N Dimentional vector.
        :param y: training data target value -----> 1 Dimentional array.
        """
        # change the data to 
        X_poly = PolynomialFeature(X, degree=self.degree)
        return super().train(X_poly, y)
    
    def predict(self, test_X):
        """
        :param test_X: feature values to predict.
        """
        test_X_poly = PolynomialFeature(test_X, degree=self.degree)
        return super().predict(test_X_poly)

In [19]:
#define the parameters
param = {
    "lamda": 0.1,
    "degree" : 2,
    "learning_rate" : 0.1,
    "iteration" : 100,
}
print("="*100)
polynomial_reg = PolynamialRidgeRegression(**param)

# Train the model.
polynomial_reg.train(X, y) 

# Predict the values.
y_pred = polynomial_reg.predict(X)

#Root mean square error.
score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

The Cost function for the iteration 10----->4178.872832133191 :)
The Cost function for the iteration 20----->2887.989505020741 :)
The Cost function for the iteration 30----->2785.6247039737964 :)
The Cost function for the iteration 40----->2777.471815365709 :)
The Cost function for the iteration 50----->2776.819294060092 :)
The Cost function for the iteration 60----->2776.7666829082946 :)
The Cost function for the iteration 70----->2776.7623662294877 :)
The Cost function for the iteration 80----->2776.761991761519 :)
The Cost function for the iteration 90----->2776.761953080877 :)
The Cost function for the iteration 100----->2776.761947221511 :)
The r2_score of the trained model 0.9718297887794873


# Supervised Machine Learning models scratch series....
you can also check....

- 1) Linear Regression         ---> https://www.kaggle.com/ninjaac/linear-regression-from-scratch
- 2) Lasso Regression          ---> https://www.kaggle.com/ninjaac/lasso-ridge-regression 
- 3) Ridge Regression          ---> https://www.kaggle.com/ninjaac/lasso-ridge-regression 
- 4) ElasticNet Regression     ---> https://www.kaggle.com/ninjaac/elasticnet-regression 
- 5) Polynomail Regression     ---> https://www.kaggle.com/ninjaac/polynomial-and-polynomialridge-regression (Same Notebook you are looking now)
- 5) PolynomailRidge Regression---> https://www.kaggle.com/ninjaac/polynomial-and-polynomialridge-regression (Same Notebook you are looking now)